In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!apt install unzip
!unzip -q "/content/drive/My Drive/fruits"

In [ ]:
# libraries needed to run model 
import numpy as np
import pandas as pd
import os
from glob import glob
import matplotlib.pyplot as plt
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix

In [ ]:
# loading in the data 
train_path = '../content/fruits-360_dataset/fruits-360/Training'
test_path = '../content/fruits-360_dataset/fruits-360/Test'

In [ ]:
vgg = VGG16(input_shape=[100,100,3], weights='imagenet', include_top=False)

for layer in vgg.layers:
  layer.trainable = False

x = Flatten()(vgg.output)
prediction = Dense=(120, activation='softmax')(x)

In [ ]:
# Create Model
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

# Configure model
model.compile(
  loss='categorical_crossentropy',
  optimizer='rmsprop',
  metrics=['accuracy'])

In [ ]:
# Create an instance of ImageDataGenerator
gen = ImageDataGenerator(
  rotation_range=20,
  width_shift_range=0.1,
  height_shift_range=0.1,
  shear_range=0.1,
  zoom_range=0.2,
  horizontal_flip=True,
  vertical_flip=True,
  rescale=1./255,  
  preprocessing_function=preprocess_input,
  validation_split=0.2)

In [ ]:
# Create generators for training and validation by splitting the training file into 80% training and 20% validation set
train_generator = gen.flow_from_directory(
  train_path,
  target_size=[100,100],
  shuffle=True,
  batch_size=120,
  subset='training')

valid_generator = gen.flow_from_directory(
  train_path,
  target_size=[100,100],
  shuffle=False,
  batch_size=120,
  subset='validation')

# Fitting the VGG16 model 
r = model.fit_generator(
  train_generator,
  validation_data=valid_generator,
  epochs=10,
  steps_per_epoch=100,
  validation_steps=80)

In [ ]:
# Plot the train and validation loss
plt.plot(r.history['loss'], label='train loss')
plt.plot(r.history['val_loss'], label='val loss')
plt.legend()
plt.title('Train and Validation Loss')
plt.show()

# Plot the train and validation accuracies
plt.plot(r.history['acc'], label='train acc')
plt.plot(r.history['val_acc'], label='val acc')
plt.legend()
plt.title('Train and Validation Accuracy')
plt.show()

In [ ]:

print("Final training accuracy = {}".format(r.history["acc"][-1]))
print("Final validation accuracy = {}".format(r.history["val_acc"][-1]))